# Lab 4: What was that sound?

### Instructions:
- perform a fresh `restart & run all` before submitting the `.ipynb` to [gradescope](https://www.gradescope.com/courses/478298)
- [lab rubric](https://course.ccs.neu.edu/ds2500/admin_syllabus.html?highlight=rubric#weekly-lab-ds-2501)
- work in groups of 2-5
- be collaborative and kind
    - ask questions of others
    - invite questions from others
- each student will submit their own lab file
- please do not share code files 
    - however, unlike HW, you're welcome to look at each other's ungraded work

# Introduction

In this lab, you'll be given about 150 recordings of me saying a few different names (mostly they're pets I've had in my life, [here is Zeke, in all his glory](https://photos.app.goo.gl/6TA1mZh7dTv5cUWn6)).  Your goal is to build a classifier which can distinguish which name I'm saying.

### Notes / hints:
- avoid digging into the `lab_helper.py` file until the final part.  
- be sure that you're comfortable with precisely what `x` and `y` are below
    - they're arrays in sklearn format
    - try playing a few noises
        - see code cell just before Part A
- [intro video](https://northeastern.zoom.us/rec/share/BmqC9mSlPwMVjR5lWFyw4_N-ws6QY9VfrVbCQILWpiKcdh8XnaqkfbP89DqYBUJo.ih5qUmin8uiyGOS5)

In [ ]:
import pathlib

from lab_helper import get_df_wav, load_wav, get_xy_array

folder = pathlib.Path('.') / 'sound_wav'
df_wav = get_df_wav(folder)

df_wav.head()

In [ ]:
from IPython.display import display, Audio

def play(x, rate):
    """ plays a noise via jupyter
    
    credit: Dheeraj (chillamcharla.d@northeastern.edu), thank you!
    
    Args:
        x (np.array): 1d audio signal
        rate (int): sampling rate
    """
    if x.ndim == 1:
        # mono
        x_tup = x,
    elif x.ndim == 2:
        # stereo
        x_tup = x[:, 0], x[:, 1]
    else:
        raise AttributeError('invalid x.ndim')
        
    # play audio
    display(Audio(x_tup, rate=rate))

In [ ]:
# sampling rate of audio (error thrown if not this value)
rate = 44100

# we discard this much audio (seconds) from start of every clip
time_trim_start = .2

# clips are truncated / extended with silence to ensure they're this long (seconds)
time_total = 1.3

x, y, file_list = get_xy_array(df_wav, rate=rate, time_trim_start=time_trim_start, time_total=time_total)

In [ ]:
x.shape

In [ ]:
y

In [ ]:
# change the index of the sample below and re-run this cell to hear different noises
play(x=x[-1, :], rate=rate)

# Part A: Classification

Build a k-NN classifier which classifies what the speaker is saying (either 'annie', 'fleck', 'linda', 'matt' or 'zeke').  Produce a complete set of cross validated estimates `y_pred`.

Choose the k of k-NN arbitrarily here, we'll optimize later.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

k = 10
x_feat_list = []
y_feat = "word"

x = df_wav.loc[:, x_feat_list].values

y_true = df_wav.loc[:, y_feat].values

knn_classifier = KNeighborsClassifier(n_neighbors = k)

knn_classifier.fit(x, y_true)

y_pred = knn_classifier.predict(x)

# Part B: Confusion Matrix

Plot a confusion matrix which shows the performance of our initial classifier.  In a sentence or two, summarize what the confusion matrix shows is happening.  

Our initial classifier, as per usual, may not work too well.  Thanks ok!  Real ML applications work like this:
1. try to build something
1. see if it works
1. if it doesn't work:
    1. find out why
    1. fix it
    1. repeat from step 2 above

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

conf_mat = confusion_matrix(y_true = y_true, y_pred = y_pred)
conf_mat_disp = ConfusionMatrixDisplay(conf_mat, display_labels=np.unique(y_true))

sns.set(font_scale = 2)
conf_mat_disp.plot()
plt.suptitle(".wav classification")
plt.gcf().set_size_inches(7, 7)
plt.grid(False)

# Part C: Tuning up our classifier and data

[part c video](https://northeastern.zoom.us/rec/share/kPUjJd-2-lw1skuqw5Q-4iBHHWQFBSNtOMNvLSHvmlPBkM0-eB_PHp84SJFwtvGe.JH1ylku-jldDQGMq)

The classifier in Part B didn't work too well ... did it?  Usually our ML approaches will not do their best right away, the challenge is to understand why and see what we can do to tune it up.  The plot below shows each noise of every category, with unique samples overlaid in different colors:

<img src="https://i.ibb.co/MC2CrNp/noise-initial.png" width=800>


Examining the initial plots of our data, we see that there's a few problems with the input data.  Fix these problems:
1. The volume differs per sample
    - This is not appropriate.  Implicitly, just because one noise is louder or queiter shouldn't make it more or less likely to be a particular word.
    - How can we make the volume the same for all samples?  
        - Hint: see scale normalization notes
2. Noises, even from a single class, all start at different times!
    - Due to inconsistencies in my ability to record, the noise "Fleck" starts a bit earlier or later in the recording (notice that different samples of "Fleck" are shifted left / right from each other in the image below)
    - This is not appropriate. Ideally the first utterance of the "F" noise should occur at the same index in each recording
    - Is there some way we can "chop-off" the beggining of each noise, up to the point where it first begins?  
        - Doing so would "align" our noises so we don't have this shifted behavior below.
        - I found [np.where](https://numpy.org/doc/stable/reference/generated/numpy.where.html) helpful, other approachs work too
<img src="https://i.ibb.co/0VynmD4/fleck-detail-no-time-align.png" width=400>

Hint for these first two issues: I'd suggest modifying the function `load_wav()` in `lab_helper.py`.  You can submit this alongside the lab to show your work.

3. There's a few parameters we can tune to make sure our classifier is best:
    - the "k" of k nearest neighbors
        - hint: see day13 ica2 for an example
    - the total length of time each sample runs for
    
To see if you've made progress in resolving the problems above, rebuild a cross validated confusion matrix and see if your performance improves.  (Hint: if you resolve the first two problems above the classifier performs reasonably well)